<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Credit Card Fraud Detection - Data Cleansing and Feature Engineering Pipeline</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
This is a demonstration of Vantage capabilities for functional demos e.g.
    <li style = 'font-size:16px;font-family:Arial'> Data Cleansing Functions - like GetFutileColumns, GetRowsWithMissingValues, SimpleImputeFit and SimpleImputeTransform </li>
    <li style = 'font-size:16px;font-family:Arial'> Feature Engineering Functions - OrdinalEncoding, Onehotencoding </li>
</p>
<br>
<p style = 'font-size:16px;font-family:Arial'>
In a typical Data Science project there are multiple preprocessing steps involved to process the raw incoming data before it can actually be used in a model for predictions. On an estimate about 70-80% of the time and effort goes into the pre-processing steps. With the help of Vantage's in Db functions we can perform these functions very effectively and at scale.
(add data description)
</p>  

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Accessing the Data </b> </p>
<p style = 'font-size:16px;font-family:Arial'>These demos will work either with foreign tables accessed from Cloud Storage via NOS or you may import the tables to your machine. If you import data for multiple demos, you may need to use the Data Dictionary "Manage Your Space" routine to cleanup tables you no longer need.     
    
<p style = 'font-size:16px;font-family:Arial'>Use the link below to access the 2 options for using data from the data dictionary notebook:

[Click Here to get data for this notebook](../Data_Dictionary/Data_Dictionary.ipynb#TRNG_TDU_TD01)

[Click Here to Manage Your Space](../Data_Dictionary/Data_Dictionary.ipynb#Manage_Your_Space)

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>1. Connect to Vantage and explore the dataset</b></p>
Below command will connect to the Vantage environment.

In [1]:
%connect local

Password: ···


Success: 'local' connection established and activated for user 'demo_user'


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Access data in Vantage  </b> </p>
<p style = 'font-size:16px;font-family:Arial'>For this demo, data is already resident in Object Storage which we are accessing via ReadNOS.  Create a reference to the table, and sample the contents.  Data could just as easily reside in permanent tables, another RDBMS, or another Vantage system.</p>

In [32]:
sel top 5 * from demo_user.credit_card;

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,HOUSETYPE_MODE,OCCUPATION_TYPE,age
1,112747,0,Cash loans,F,N,0,225000,Married,0.046220000833272934,-16486,1,1,2,,Managers,45
2,129917,0,Cash loans,M,N,1,121500,Civil marriage,0.022624999284744263,-12670,1,1,3,block of flats,Laborers,34
3,110198,0,Cash loans,M,N,0,157500,Married,0.018208999186754227,-10886,1,1,2,,Laborers,29
4,267481,1,Cash loans,M,N,0,112500,Single / not married,0.024609999731183052,-19559,1,1,1,,Laborers,53
5,312486,1,Cash loans,F,N,0,85500,Married,0.030755000188946724,-21010,1,0,2,,,57


In [3]:
select count(*) from demo_user.credit_card;

,Count(*)
1,50000


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Checking data demographics  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_ColumnSummary </b>function  displays Column name, datatype and other demogarphics like count of NULLs etc for each specified input table column</p>

In [35]:
SELECT * FROM TD_ColumnSummary (
 ON credit_card AS InputTable
 USING
 TargetColumns ('[:]')
) AS dt;

,ColumnName,Datatype,NonNullCount,NullCount,BlankCount,ZeroCount,PositiveCount,NegativeCount,NullPercentage,NonNullPercentage
1,TARGET,INTEGER,50000,0,,25175,24825,0,0,100
2,FLAG_MOBIL,INTEGER,50000,0,,1,49999,0,0,100
3,FLAG_OWN_CAR,VARCHAR(10) CHARACTER SET UNICODE,50000,0,0,,,,0,100
4,SK_ID_CURR,BIGINT,50000,0,,0,50000,0,0,100
5,NAME_CONTRACT_TYPE,VARCHAR(15) CHARACTER SET UNICODE,50000,0,0,,,,0,100
6,FLAG_EMP_PHONE,INTEGER,50000,0,,7638,42362,0,0,100
7,NAME_FAMILY_STATUS,VARCHAR(50) CHARACTER SET UNICODE,50000,0,0,,,,0,100
8,AMT_INCOME_TOTAL,FLOAT,50000,0,,0,50000,0,0,100
9,CODE_GENDER,VARCHAR(10) CHARACTER SET UNICODE,50000,0,0,,,,0,100
10,CNT_FAM_MEMBERS,INTEGER,50000,0,,0,50000,0,0,100


<p style = 'font-size:16px;font-family:Arial'>In below sql we are checking the null percentage of columns</p>

In [5]:
SELECT columnname, datatype, nullpercentage FROM TD_ColumnSummary (
 ON credit_card AS InputTable
 USING
 TargetColumns ('[:]')
) AS dt
where 
nullpercentage > 0
order by 3 desc;

,ColumnName,Datatype,NullPercentage
1,HOUSETYPE_MODE,VARCHAR(50) CHARACTER SET UNICODE,53.08
2,OCCUPATION_TYPE,VARCHAR(50) CHARACTER SET UNICODE,28.582


<p style = 'font-size:16px;font-family:Arial'>As the column HOUSETYPE_MODE has more than 50% of null values we can remove this column from our model calculations.<br>
    Lets check the other varchar columns. 
OCCUPATION_TYPE also has high % of null values.</p>

<p style = 'font-size:16px;font-family:Arial'><b>TD_CATEGORICALSUMMARY </b>function   displays the distinct values and their counts for each specified input table column</p>

In [6]:
Create volatile table cateogrySummaryTable as (
SELECT * FROM TD_CATEGORICALSUMMARY (
ON credit_card as inputtable
USING
TargetColumns('CODE_GENDER'
,'NAME_CONTRACT_TYPE'
,'NAME_FAMILY_STATUS'
,'FLAG_OWN_CAR'
,'OCCUPATION_TYPE')
) AS dt)With data 
on commit preserve rows;

Success: 0 rows affected

In [29]:
select top 5* from cateogrySummaryTable;

,ColumnName,DistinctValue,DistinctValueCount
1,NAME_FAMILY_STATUS,Civil marriage,5345
2,NAME_FAMILY_STATUS,Single / not married,8004
3,OCCUPATION_TYPE,Security staff,1275
4,OCCUPATION_TYPE,Medicine staff,1289
5,OCCUPATION_TYPE,HR staff,79


<p style = 'font-size:16px;font-family:Arial'><b>TD_GETFUTILECOLUMNS </b>function displays the categorical columns which will have no effect on the model i.e if all the values are same or unique or If the count of distinct values in the columns divided by the count of the total number of rows in the input
table is greater than or equal to the threshold value</p>

In [7]:
Select * from TD_getFutileColumns(
ON credit_card as inputtable partition by any
ON cateogrySummaryTable as categorytable Dimension
USING
CategoricalSummaryColumn('ColumnName') 
ThresholdValue(0.05)
)As dt;

,ColumnName
1,FLAG_OWN_CAR


<p style = 'font-size:16px;font-family:Arial'>Here we can see that FLAG_OWN_CAR will have no effect on the model as all the values in this column are same so we can remove this column from model creation</p>

<p style = 'font-size:16px;font-family:Arial'>Let us check the values in the OCCUPATION_TYPE column to see what we can do for the NULLs in the column 
</p>

In [8]:
SELECT * FROM cateogrySummaryTable where columnname = 'OCCUPATION_TYPE' order by DistinctValueCount desc;

,ColumnName,DistinctValue,DistinctValueCount
1,OCCUPATION_TYPE,,14291
2,OCCUPATION_TYPE,Laborers,10256
3,OCCUPATION_TYPE,Sales staff,5684
4,OCCUPATION_TYPE,Core staff,4060
5,OCCUPATION_TYPE,Drivers,3570
6,OCCUPATION_TYPE,Managers,3057
7,OCCUPATION_TYPE,High skill tech staff,1647
8,OCCUPATION_TYPE,Accountants,1308
9,OCCUPATION_TYPE,Medicine staff,1289
10,OCCUPATION_TYPE,Security staff,1275


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Impute Missing Values  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_SimpleImputeFit </b>will output a table with the values that will be used to substitute the missing values<br>
    <b>TD_SimpleImputeTransform</b> will return the input data set with the missing values filled in.
Verify the NULL values have been removed.<br>
    *Note one can also use the Fit table as input to <b>TD_ColumnTransformer</b>

In [9]:
-- fit the SimpleImpute function on categorical columns
SELECT * FROM TD_SimpleImputeFit (
    ON credit_card as InputTable
    OUT VOLATILE TABLE OutputTable(impute_fit_cat_output)
    USING
    ColsForLiterals ('OCCUPATION_TYPE')
    Literals ('not provided')
) as dt;



Success: 0 rows affected

,TD_INDEX_SIMFIT,TD_TARGETCOLUMN_SIMFIT,TD_NUM_COLVAL_SIMFIT,TD_STR_COLVAL_SIMFIT,TD_ISNUMERIC_SIMFIT
1,14,OCCUPATION_TYPE,,not provided,0


In [10]:
Create volatile table occupationimputetable as (
SELECT * FROM TD_SimpleImputeTransform (
 ON credit_card as InputTable
 ON impute_fit_cat_output AS FitTable DIMENSION
) AS dt)With data 
on commit preserve rows;

Success: 0 rows affected

<p style = 'font-size:16px;font-family:Arial'>TD_SimpleImputeFit and TD_SimpleImputeTransform function works on integer columns also and we can use them for filling the missing column values based on min/max/mean/median of the values in the column</p>

<p style = 'font-size:16px;font-family:Arial'>In model creation we usually prefer Numerical inputs instead of Characters, now let us check how many distinct values we have for our character columns so that we can encode them in numerals</p>

In [11]:
SELECT columnname,count(distinctvalue) FROM cateogrySummaryTable 
group by 1 order by 2; 

,ColumnName,DistinctValue
1,FLAG_OWN_CAR,1
2,NAME_CONTRACT_TYPE,2
3,CODE_GENDER,2
4,NAME_FAMILY_STATUS,5
5,OCCUPATION_TYPE,18


<p style = 'font-size:16px;font-family:Arial'>We are not using FLAG_OWN_CAR column in model creation, for other columns we can use <b>TD_OneHotEncodingFit</b> and <b>TD_OrdinalEncodingFit</b> and transform functions to convert character categories to numerals, for that we need to check the exact values present in the columns</p>

In [12]:
SELECT columnname,distinctvalue FROM cateogrySummaryTable where columnname in 
('CODE_GENDER', 'NAME_CONTRACT_TYPE')
order by 1; 

,ColumnName,DistinctValue
1,CODE_GENDER,M
2,CODE_GENDER,F
3,NAME_CONTRACT_TYPE,Revolving loans
4,NAME_CONTRACT_TYPE,Cash loans


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Feature Engineering Functions  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_OneHotEncodingFit </b>outputs a table of attributes and categorical values to input to <b>TD_OneHotEncodingTransform </b> which encodes them as one-hot numeric vectors.</p>

In [13]:
CREATE VOLATILE TABLE onehotencodingfit_genderoutput AS (
 SELECT * FROM TD_OneHotEncodingFit (
 ON credit_card AS InputTable
 USING
 TargetColumn ('code_gender')
 OtherColumnName ('other')
 CategoricalValues ('M', 'F')
 IsInputDense ('true')
 ) AS dt
) WITH DATA
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

<p style = 'font-size:16px;font-family:Arial'>We can check how the fit table looks like</p>

In [14]:
select * from onehotencodingfit_genderoutput;

,CODE_GENDER,CODE_GENDER_M,CODE_GENDER_F,CODE_GENDER_other
1,,M,F,


<p style = 'font-size:16px;font-family:Arial'>We will create a similar fit table for the other categorical column</p>

In [15]:
CREATE VOLATILE TABLE onehotencodingfit_contractoutput AS (
 SELECT * FROM TD_OneHotEncodingFit (
 ON credit_card AS InputTable
 USING
 TargetColumn ('NAME_CONTRACT_TYPE')
 OtherColumnName ('other')
 CategoricalValues ('Revolving loans', 'Cash loans')
 IsInputDense ('true')
 ) AS dt
) WITH DATA
ON COMMIT PRESERVE ROWS;


Success: 0 rows affected

In [16]:
select * from onehotencodingfit_contractoutput;

,NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_other
1,,Revolving loans,Cash loans,


<p style = 'font-size:16px;font-family:Arial'>For categorical columns which have many values we can use <b>TD_OrdinalEncoding</b> instead</p>

In [30]:
SELECT * FROM TD_OrdinalEncodingFit (
 ON credit_card AS InputTable
 OUT table outputtable (ordinalencodingfit_familyoutput)
 USING
 TargetColumn ('NAME_FAMILY_STATUS')
 DefaultValue (-1)
) as dt;

Success: 0 rows affected

,NAME_FAMILY_STATUS,TD_VALUE_ORDFIT
1,Civil marriage,0
2,Married,1
3,Separated,2
4,Single / not married,3
5,Widow,4
6,TD_OTHER_CATEGORY,-1


<p style = 'font-size:16px;font-family:Arial'>Similarly we will use TD_OrdinalEncodingFit for OCCUPATION_TYPE also. Note that we have used occupationimputetable which is the output of Impute function applied on Occupation_Type column as we want the Null values in a separate category</p>

In [18]:
SELECT * FROM TD_OrdinalEncodingFit (
 ON occupationimputetable AS InputTable
 OUT volatile table outputtable (ordinalencodingfit_occupationoutput)
 USING
 TargetColumn ('OCCUPATION_TYPE')
 DefaultValue (-1)
) as dt;

Success: 0 rows affected

,OCCUPATION_TYPE,TD_VALUE_ORDFIT
1,TD_OTHER_CATEGORY,-1
2,Accountants,0
3,Cleaning staff,1
4,Cooking staff,2
5,Core staff,3
6,Drivers,4
7,High skill tech staff,5
8,HR staff,6
9,IT staff,7
10,Laborers,8


In [ ]:
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Checking data demographics  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_ColumnSummary </b>function  displays Column name, 
datatype and other demogarphics like count of NULLs etc for each specified input table column</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> TD_ColumnTransformer  </b> </p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ColumnTransformer function transforms the entire dataset in a single operation. You only need
to provide the FIT tables to the function, and the function runs all transformations that you require in a
single operation</p>

<p style = 'font-size:16px;font-family:Arial'>Let us put all the fit tables we have created and transform the dataset</p>

In [31]:
SELECT *
    FROM TD_ColumnTransformer(
 ON credit_card AS inputtable
 ON  impute_fit_cat_output AS SImpleImputeFitTable DIMENSION   
 ON onehotencodingfit_genderoutput AS ONehotencodingfittable dimensiON
 ON onehotencodingfit_contractoutput AS ONehotencodingfittable dimensiON  
 ON ordinalencodingfit_familyoutput AS OrdinalEncodingFitTable dimensiON
 ON ordinalencodingfit_occupationoutput AS OrdinalEncodingFitTable dimensiON  
)
AS dt
;


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,HOUSETYPE_MODE,OCCUPATION_TYPE,CODE_GENDER_M,CODE_GENDER_F,CODE_GENDER_other,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_other
1,312486,1,Cash loans,F,N,0,85500,1,0.030755000188946724,-21010,1,0,2,,12,0,1,0,0,1,0
2,123881,0,Cash loans,F,N,0,135000,1,0.007329999934881926,-18735,1,1,2,block of flats,12,0,1,0,0,1,0
3,447501,1,Cash loans,M,N,0,67500,3,0.0014169999631121755,-10044,1,1,1,,15,1,0,0,0,1,0
4,143600,1,Cash loans,F,N,0,90000,0,0.02280000038444996,-9981,1,1,2,,1,0,1,0,0,1,0
5,267481,1,Cash loans,M,N,0,112500,3,0.024609999731183052,-19559,1,1,1,,8,1,0,0,0,1,0
6,107180,0,Cash loans,M,N,0,81000,3,0.03132899850606918,-8803,1,1,1,,8,1,0,0,0,1,0
7,118314,0,Cash loans,F,N,0,121500,1,0.006008000113070011,-15650,1,1,2,block of flats,5,0,1,0,0,1,0
8,126899,0,Cash loans,F,N,2,180000,1,0.025164000689983368,-11978,1,1,4,block of flats,15,0,1,0,0,1,0
9,112747,0,Cash loans,F,N,0,225000,1,0.046220000833272934,-16486,1,1,2,,10,0,1,0,0,1,0
10,191623,1,Cash loans,F,N,0,112500,1,0.005313000176101923,-13794,1,1,2,,15,0,1,0,0,1,0


In [22]:
Create table Trasformed_data as(
SELECT SK_ID_CURR,"TARGET",CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE
    ,DAYS_BIRTH,FLAG_MOBIL,    FLAG_EMP_PHONE,CNT_FAM_MEMBERS,OCCUPATION_TYPE,CODE_GENDER_M,CODE_GENDER_F,
    "NAME_CONTRACT_TYPE_REVOLVING LOANS","NAME_CONTRACT_TYPE_CASH LOANS" 
    FROM TD_ColumnTransformer(
 ON credit_card AS inputtable
 ON  impute_fit_cat_output AS SImpleImputeFitTable DIMENSION   
 ON onehotencodingfit_genderoutput AS ONehotencodingfittable dimensiON
 ON onehotencodingfit_contractoutput AS ONehotencodingfittable dimensiON  
 ON ordinalencodingfit_familyoutput AS OrdinalEncodingFitTable dimensiON
 ON ordinalencodingfit_occupationoutput AS OrdinalEncodingFitTable dimensiON  
)
AS dt
) WITH DATA
;


Success: 0 rows affected

In [23]:
select top 10* from Trasformed_data;

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,OCCUPATION_TYPE,CODE_GENDER_M,CODE_GENDER_F,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans
1,177471,1,0,112500,0,0.014519999735057354,-16495,1,1,2,2,0,1,0,1
2,219927,1,2,135000,0,0.011703000403940678,-12068,1,1,4,15,0,1,0,1
3,298803,1,0,180000,2,0.022624999284744263,-17559,1,1,1,8,1,0,0,1
4,118783,0,1,135000,1,0.01968899928033352,-11061,1,1,3,8,1,0,0,1
5,107649,0,0,72000,1,0.03579200059175491,-16382,1,1,2,15,0,1,0,1
6,116234,0,1,675000,1,0.01000600028783083,-13710,1,1,3,10,0,1,0,1
7,203226,1,0,405000,1,0.03579200059175491,-11123,1,1,2,4,1,0,0,1
8,385795,1,0,63000,3,0.00886599998921156,-8871,1,1,1,12,0,1,0,1
9,118314,0,0,121500,1,0.006008000113070011,-15650,1,1,2,5,0,1,0,1
10,447501,1,0,67500,3,0.0014169999631121755,-10044,1,1,1,15,1,0,0,1


<p style = 'font-size:16px;font-family:Arial'>Here we can see that we have now transformed our data and we can further use this data in the model creation.
</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Clean up</b> </p>

In [24]:
drop table Trasformed_data;


Success: 29 rows affected

In [26]:
drop table ordinalencodingfit_familyoutput;

Success: 15 rows affected

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Conslusion </b> </p>
<p style = 'font-size:16px;font-family:Arial'>In this notebook we have seen some of the Teradata Vantage's new inDb functions for data cleansing, data exploration and feature engineering. Many of these functions can be applied in one go using the TD_COLUMNTRANSFORM function which gives is approx 30% faster than serial processing</p>

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
        <li>Teradata Analytic Function Reference:
        <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Analytics-Database-Analytic-Functions-Overview'>
        https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Analytics-Database-Analytic-Functions-Overview</a></li>
  
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>